In [1]:
import glob
import os
import random
import shutil

In [2]:
image_names = glob.glob("images/*.jpg")

image_names = [x.replace('images/', '').replace('.jpg', '') for x in image_names]

In [3]:
val_ratio = 0.1
test_ratio = 0.15 # 15%
train_ratio = 1 - val_ratio - test_ratio



random.shuffle(image_names)

num_train = round(len(image_names) * train_ratio)
num_val = round(len(image_names) * val_ratio)


train_images = image_names[:num_train]
val_images = image_names[num_train: num_train + num_val]
test_images = image_names[num_train + num_val:]

In [4]:
print('Number of training images: ', len(train_images))
print('Number of validation images: ', len(val_images))
print('Number of test images: ', len(test_images))


In [5]:
## Create directories

if os.path.exists("train_dataset"):
    shutil.rmtree('train_dataset')

os.makedirs('train_dataset/Annotations')
os.makedirs('train_dataset/JPEGImages')


In [6]:
# copy images:
for name in train_images:
    shutil.copyfile('images/' + name + '.jpg', 'train_dataset/JPEGImages/' + name + '.jpg')
    

In [7]:
# train file names to txt
#with open('indycar_dataset/ImageSets/Main/train.txt', 'w') as train_file:
#    train_file.writelines("%s\n" % name for name in train_images)
#    train_file.truncate(train_file.tell()-1)
#    
# # val file names to txt
#with open('indycar_dataset/ImageSets/Main/val.txt', 'w') as val_file:
#    val_file.writelines("%s\n" % name for name in val_images)
#    val_file.truncate(val_file.tell()-1)

In [8]:
# test file names to txt
#with open('indycar_dataset/ImageSets/Main/test.txt', 'w') as test_file:
#    test_file.writelines("%s\n" % name for name in test_images)
#    test_file.truncate(test_file.tell()-1)

In [9]:
## Annotations fix and save it into target directory. No need to copy annotations since this saves to target dir.

from xml.etree import ElementTree as et

def fix_annotations_save(source_dir, file_name, target_dir):
    
    # some files have problematic annotations. first fix them
    # Read in the file
    with open(source_dir + file_name + '.xml') as file :
        filedata = file.read().replace('<path', '<path>').replace('annotations', 'annotation')
    # Write the file out again
    with open(source_dir + file_name + '.xml', 'w') as file:
        file.write(filedata)

        
        
    tree = et.parse(source_dir + file_name + '.xml')
    root = tree.getroot()

    
    # some old files have unneccessary tags, and these give error. Remove them
    for path in root.findall('path'):
        root.remove(path)
    for source in root.findall('source'):
        root.remove(source)
    
    for elem in root.getiterator():
        # some cars labelled as dogs. Fix them
        if elem.text == "dog":
            elem.text = elem.text.replace('dog', 'car')
        if elem.text == "images":
            elem.text = elem.text.replace('images', 'JPEGImages')
        # some files start with annotation tag.This should be annotations
        if elem.text == "annotations":
            elem.text = elem.text.replace('annotations', 'annotation')
        
    tree.write(target_dir + file_name + '.xml')
    


for name in train_images:
    fix_annotations_save('annotations/', name, 'train_dataset/Annotations/')

print('done')


done


## Validation data

In [10]:
## Create directories

if os.path.exists("val_dataset"):
    shutil.rmtree('val_dataset')

os.makedirs('val_dataset/Annotations')
os.makedirs('val_dataset/JPEGImages')


# copy images:
for name in val_images:
    shutil.copyfile('images/' + name + '.jpg', 'val_dataset/JPEGImages/' + name + '.jpg')
    

for name in val_images:
    fix_annotations_save('annotations/', name, 'val_dataset/Annotations/')

## Test images

In [16]:
## Create directories

if os.path.exists("test_dataset"):
    shutil.rmtree('test_dataset')

os.makedirs('test_dataset/Annotations')
os.makedirs('test_dataset/JPEGImages')


# copy images:
for name in test_images:
    shutil.copyfile('images/' + name + '.jpg', 'test_dataset/JPEGImages/' + name + '.jpg')
    

for name in test_images:
    fix_annotations_save('annotations/', name, 'test_dataset/Annotations/')

In [12]:
# Assumes that TensorFlow is already installed


#dependencies
#apt-get install -y protobuf-compiler python-pil python-lxml python-tk
#pip install --user Cython
#pip install --user contextlib2



In [17]:
!./jupyter_create_tfrecord.sh

W1003 14:23:00.889311 140151796623168 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1003 14:23:01.007325 140151796623168 deprecation_wrapper.py:119] From /home/sakkas/Downloads/indycar_data_prepare/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1003 14:23:01.011741 140151796623168 deprecation_wrapper.py:119] From /home/sakkas/Downloads/indycar_data_prepare/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

Running tests under Python 3.7.3: /usr/bin/python3
[ RUN      ] ModelBuilderTest.t

W1003 14:23:07.597631 140512087762752 deprecation_wrapper.py:119] From models/research/object_detection/dataset_tools/create_pascal_tf_record.py:174: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W1003 14:23:07.598032 140512087762752 deprecation_wrapper.py:119] From models/research/object_detection/dataset_tools/create_pascal_tf_record.py:146: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W1003 14:23:07.598230 140512087762752 deprecation_wrapper.py:119] From /home/sakkas/Downloads/indycar_data_prepare/models/research/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

I1003 14:23:07.600780 140512087762752 create_pascal_tf_record.py:159] On image 0 of 464
/home/sakkas/Downloads/indycar_data_prepare/models/research/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use spec